In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#_______________________________________________
# Charegement des libraries Keras 
#
from tensorflow.keras.models import Sequential
### Conv2d pour des photos - 2D si Video ou coleur alors 3D 
from  tensorflow.keras.layers import Conv2D
### Pour la phse de Pooling 2D si  couleur ou Videos alors 3D
from  tensorflow.keras.layers import MaxPooling2D
### Etape 3 - Applatir dans un vector Vertical 
from  tensorflow.keras.layers import Flatten
### Dense pour ajouter des couches connectées.
from  tensorflow.keras.layers import Dense
### Droput si necessaire
from tensorflow.keras.layers import MaxPool2D,Dropout
### import Metrics
from  tensorflow.keras.metrics import *
#import keras

In [ ]:
#from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, Adamax, RMSprop
from  tensorflow.keras.metrics import *
from tensorflow.keras import Model
#from keras import backend as 
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

## With Regularization
from tensorflow.keras import regularizers, optimizers

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.layers import Activation

In [ ]:
import keras 
from keras import backend as K



In [ ]:
keras.__version__

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Prepare Dataset

In [ ]:
# Name Label Dictionary
name_label_dict = {
0:  "Nucleoplasm", 
1:  "Nuclear membrane",   
2:  "Nucleoli",   
3:  "Nucleoli fibrillar center" ,  
4:  "Nuclear speckles"   ,
5:  "Nuclear bodies"   ,
6:  "Endoplasmic reticulum",   
7:  "Golgi apparatus"   ,
8:  "Intermediate filaments",   
9:  "Actin filaments"   ,
10:  "Microtubules"   ,
11:  "Mitotic spindle"   ,
12:  "Centrosome" , 
13:  "Plasma membrane",   
14:  "Mitochondria"   ,
15:  "Aggresome"   ,
16:  "Cytosol",  
17:  "Vesicles and punctate cytosolic patterns",   
18:  "Negative" 
}

In [ ]:
#Load Train Dataset 
path_to_train = '../input/hpa-single-cell-image-classification/train/'
data = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')

In [ ]:
data.head(), data.shape

In [ ]:
def build_labels(df):
    # dataframe with column for each class
    labels = list()
    
    for index, sample in df.iterrows():
        # zero out class array
        label = [0] * 19
        
        # for each class found in training sample, flip lablel value to one
        for cls in sample['Label'].split('|'):
            label[int(cls)] = 1

        # Append label to list
        labels.append( np.array(label) )

    return np.vstack(labels)

In [ ]:
df_train_labels = build_labels(data)
df_train_labels.shape

In [ ]:
#____________________________________________________
# Add full path to images files
data['image_path']=path_to_train + data['ID']

In [ ]:
## Join train_labels 
## Just Testing Green "Cells"
df_train_green = pd.DataFrame(data['image_path']).join(pd.DataFrame(df_train_labels))

In [ ]:
df_train_green.head()

In [ ]:
df_train_green['image_path'] = df_train_green['image_path']+'_green.png'

In [ ]:
pd.set_option('display.max_colwidth', None)
df_train_green.head()

In [ ]:
classes = []
for key, value in name_label_dict.items():
    print(value)
    classes.append(value)
print(classes)

In [ ]:
df_train_green.columns = ['path']+classes
df_train_green.head(2)

In [ ]:
## Save dataset
## df_train_green.to_csv('./df_train_green.csv', index=False)

In [ ]:
## Load dataset
df_train_green = pd.read_csv('../input/df-train-green-csv/df_train_green.csv') ## Multioutpt Classes 19 Columns
df_train_green.head()

In [ ]:
classes,type(classes)

In [ ]:
classes = ['Nucleoplasm', 'Nuclear membrane', 'Nucleoli', 'Nucleoli fibrillar center', 
           'Nuclear speckles', 'Nuclear bodies', 'Endoplasmic reticulum', 'Golgi apparatus', 
           'Intermediate filaments', 'Actin filaments', 'Microtubules', 'Mitotic spindle', 
           'Centrosome', 'Plasma membrane', 'Mitochondria', 'Aggresome', 'Cytosol', 
           'Vesicles and punctate cytosolic patterns', 'Negative']
type(classes)

In [ ]:
output_list=[]
loss_list = []
for p in enumerate(classes):
    #print(p[0])
    print('output'+str(p[0])+',')
    output_list.append('output'+str(p[0]))
    loss_list.append("binary_crossentropy")
output_list, loss_list

In [ ]:

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   #validation_split=0.3, ## Not Yet Compiled - 
                                   horizontal_flip = True)

### pour le Test set
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_dataframe(dataframe=df_train_green[:200],
                                                 x_col='path',
                                                 y_col= classes,
                                                 #directory='../input/hpa-single-cell-image-classification/train',
                                                 target_size = (128, 128),
                                                 batch_size = 256,
                                                 validation_split = .3,
                                                 seed = 7,
                                                 class_mode = 'multi_output')  ## if "raw" need wrapper_generator function 

In [ ]:
### 

test_set = test_datagen.flow_from_dataframe(dataframe=df_train_green[200:300],
                                                 x_col='path',
                                                 y_col= classes,
                                            #directory='../input/hpa-single-cell-image-classification/train'
                                            #imagepath_test,
                                            target_size = (128, 128),
                                            batch_size = 256,
                                            seed = 7,
                                            shuffle=False,  ## Si nous voulons utiliser X_test dans Matrice Conf.
                                            #index_array = None,  ## Si nous voulons utiliser X_test dans Matrice Conf.
                                            class_mode = 'multi_output')  ## if "raw" need wrapper_generator function 

In [ ]:
inp = Input(shape = (128,128,3))
x = Conv2D(32, (3, 3), padding = 'valid')(inp)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), padding = 'valid')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
output0 = Dense(1, activation = 'sigmoid')(x)
output1 = Dense(1, activation = 'sigmoid')(x)
output2 = Dense(1, activation = 'sigmoid')(x)
output3 = Dense(1, activation = 'sigmoid')(x)
output4 = Dense(1, activation = 'sigmoid')(x)
output5 = Dense(1, activation = 'sigmoid')(x)
output6 = Dense(1, activation = 'sigmoid')(x)
output7 = Dense(1, activation = 'sigmoid')(x)
output8 = Dense(1, activation = 'sigmoid')(x)
output9 = Dense(1, activation = 'sigmoid')(x)
output10 = Dense(1, activation = 'sigmoid')(x)
output11 = Dense(1, activation = 'sigmoid')(x)
output12 = Dense(1, activation = 'sigmoid')(x)
output13 = Dense(1, activation = 'sigmoid')(x)
output14 = Dense(1, activation = 'sigmoid')(x)
output15 = Dense(1, activation = 'sigmoid')(x)
output16 = Dense(1, activation = 'sigmoid')(x)
output17 = Dense(1, activation = 'sigmoid')(x)
output18 = Dense(1, activation = 'sigmoid')(x)

In [ ]:
model = Model(inp,[output0,
output1,
output2,
output3,
output4,
output5,
output6,
output7,
output8,
output9,
output10,
output11,
output12,
output13,
output14,
output15,
output16,
output17,
output18])


In [ ]:
import tensorflow as tf

In [ ]:
rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 2, verbose = 0, 
                            min_delta = 1e-4, mode = 'max')
es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 2, mode = 'max', 
                       baseline = None, restore_best_weights = True, verbose = 0)
ckp = ModelCheckpoint('./model_1.hdf5', monitor = 'val_AUC', verbose = 0, 
                        save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
model.compile(optimizer = Adam(lr = 0.001, decay = 1e-6),
              loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 1e-3), 
              #metrics = [BinaryAccuracy(name='binary_accuracy', dtype=None, threshold=0.5), Precision(name='precision'), Recall(name='recall')] 
              metrics = [tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'), tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'), \
                         Precision(name='precision'), Recall(name='recall') ] 
              #metrics = ['categorical_accuracy', 'accuracy'],
             )

In [ ]:
# Show a summary of the model. Check the number of trainable parameters
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
# plot the autoencoder
plot_model(model, 'HPA-model1.png', show_shapes=True)

In [ ]:
## IF "raw" in 
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(19)])

In [ ]:

STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
model.fit(generator_wrapper(training_set),
                  #(np.asarray(training_set).astype("float32")),
                    #steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=generator_wrapper(test_set),
                    #batch_size=4096, 
                    #validation_steps=STEP_SIZE_VALID,
                    epochs=3,
                    callbacks = [rlr,ckp,es],
                    #verbose=2
         )

In [ ]:
STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
model.fit(training_set,
                  #(np.asarray(training_set).astype("float32")),
                    #steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    #batch_size=4096, 
                    #validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks = [rlr,ckp,es],
                    #verbose=2
         )

In [ ]:
model_name = 'HPA-model1-green-data'
print(model_name)
model.save('./'+model_name+'.hdf5')

In [ ]:
%%time
test_set.reset() ## To get the output in same oder as y_test
pred=model.predict(test_set,
verbose=1)

In [ ]:
pred[:1]

In [ ]:
pd.DataFrame(np.asarray(pred).reshape(100,-1))

In [ ]:
df_train_green.iloc[200:206,1:]

In [ ]:
(np.asarray(pred).reshape(100,-1)==).all()

In [ ]:
len([[0.24326178],
[0.24545057],
[0.18905358],
[0.22454323],
[0.17228684],
[0.23446988],
[0.23484272],
[0.19753459],
[0.26074097],
[0.2010675 ],
[0.10761576],
[0.22031462],
[0.23193006],
[0.22713315],
[0.11388983],
[0.23554248],
[0.1785873 ],
[0.25215697],
[0.18093413],
[0.1990953 ],
[0.20176446],
[0.19727392],
[0.2491296 ],
[0.11087943],
[0.22272176],
[0.21852216],
[0.207871  ],
[0.1856019 ],
[0.2678131 ],
[0.20906556],
[0.27924448],
[0.27829096],
[0.13388641],
[0.2648097 ],
[0.24399373],
[0.24952464],
[0.157573  ],
[0.23329581],
[0.23566657],
[0.27109227],
[0.22023298],
[0.21330912],
[0.28557703],
[0.17392349],
[0.17297237],
[0.24998474],
[0.24437207],
[0.06390683],
[0.20857255],
[0.23521458]])

In [ ]:
def clean_prediction(prediction):
    prediction = prediction.copy()
    for batch_index in range(len(prediction)):
        for class_index in range(len(prediction[batch_index])):
            prediction[batch_index][class_index] = 1 if prediction[batch_index][class_index] >= 0.4 else 0
    return np.array(prediction).astype(np.int) 

In [ ]:
clean_pred = clean_prediction(pred)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.DataFrame(clean_pred.reshape(100,19))

In [ ]:
(clean_pred.reshape(100,19) == df_train_green.iloc[200:300,1:].values).all()

## #Only 1 Output

In [ ]:
## testing 1 output for change in classe mode from raw to multi_output
inp = Input(shape = (128,128,3))
x = Conv2D(32, (3, 3), padding = 'valid')(inp)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), padding = 'valid')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
output0 = Dense(1, activation = 'sigmoid')(x)

In [ ]:
## testing for change in classe mode from raw to multi_output
model = Model(inp,output0)

In [ ]:
STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
model.fit(training_set,
                  #(np.asarray(training_set).astype("float32")),
                    #steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    #batch_size=4096, 
                    #validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks = [rlr,ckp,es],
                    #verbose=2
         )

In [ ]:
test_set.reset()
pred=model.predict(test_set,
verbose=1)

In [ ]:
pred